# 30 — Generate Picks

Convierte predicciones a Prob(Over) y calcula edge vs. línea.

In [ ]:
import pandas as pd, numpy as np
from src.models.predict import predict
from scipy.stats import norm

df = predict('data/processed/player_games.parquet', 'models/artifacts/model.joblib')

# DEMO: línea fija basada en posición (ajusta con tus vegas_lines reales por jugador)
df['line'] = np.where(df['position'].isin(['RB']), 45.5, 39.5)

MAE_HIST = 12.0
df['prob_over'] = 1 - norm.cdf(df['line'], loc=df['pred_yards'], scale=MAE_HIST)

def american_to_payout(odds):
    if odds < 0: return 100/(-odds)
    return odds/100

payout = american_to_payout(-110)
df['edge'] = df['prob_over']*payout - (1-df['prob_over'])

picks = df[['player','team','opp','position','line','pred_yards','prob_over','edge']].copy()
picks = picks.query('prob_over >= 0.58 and edge >= 0.08').sort_values('edge', ascending=False)
picks.head(20)